In [1]:
import numpy as np
import pandas as pd
import keras
import eng_to_ipa as ipa

Using TensorFlow backend.


In [37]:
df = pd.read_csv('Multiple_Forms.csv', index_col = 0)
df.head()

,Author,Poem,Form
0,William Shakespeare,"From fairest creatures we desire increase,\r\n...",Sonnet
1,William Shakespeare,"When forty winters shall besiege thy brow,\r\n...",Sonnet
2,William Shakespeare,Look in thy glass and tell the face thou viewe...,Sonnet
3,William Shakespeare,"Unthrifty loveliness, why dost thou spend\r\n ...",Sonnet
4,William Shakespeare,"Those hours, that with gentle work did frame\r...",Sonnet


In [39]:
poems = df['Poem'][df.Form == 'Sonnet']
poems

0     From fairest creatures we desire increase,\r\n...
1     When forty winters shall besiege thy brow,\r\n...
2     Look in thy glass and tell the face thou viewe...
3     Unthrifty loveliness, why dost thou spend\r\n ...
4     Those hours, that with gentle work did frame\r...
                            ...                        
95    Whate'er the cost to me, with this farewell,\n...
96    If the past year were offered me again,\nWith ...
97    Nay, dear one, ask me not to leave thee yet.\n...
98    Where is the pride for which I once was blamed...
99    I sue thee not for pity on my case.\nIf I have...
Name: Poem, Length: 348, dtype: object

In [40]:
text = ''
for poem in poems:
    text += str(poem) + '\n'
len(text)

258640

In [44]:
text = text.replace("’", "'")
text = text.replace("‘", "'")
text = text.replace("“", " ")
text = text.replace("'d", "ed")
text = text.replace("(", "")
text = text.replace(")", "")
text = text.replace(" '", " ")
text = text.replace("' ", " ")
text = text.replace('"', '')
text = text.replace("--", " ")
text = text.replace(":-", " ")
text = text.replace("-:", " ")
text = text.replace(".-", " ")
text = text.replace("- ", " ")
text = text.replace(" -", " ")
text = text.replace(" –", " ")
while text.find(" . ") != -1:
    text = text.replace(" . ", " ")
while text.find("..") != -1:
    text = text.replace("..", ".")
text = text.replace("?.", "? ")
text = text.replace("!.", "! ")
text = text.replace("!-", "! ")
text = text.replace(".", " ")
text = text.replace(",", " ")

In [57]:
print(text)

From fairest creatures we desire increase 
  That thereby beauty's rose might never die 
  But as the riper should by time decease 
  His tender heir might bear his memory:
  But thou  contracted to thine own bright eyes 
  Feed'st thy light's flame with self-substantial fuel 
  Making a famine where abundance lies 
  Thyself thy foe  to thy sweet self too cruel:
  Thou that art now the world's fresh ornament 
  And only herald to the gaudy spring 
  Within thine own bud buriest thy content 
  And tender churl mak'st waste in niggarding:
        Pity the world  or else this glutton be 
        To eat the world's due  by the grave and thee 
When forty winters shall besiege thy brow 
  And dig deep trenches in thy beauty's field 
  Thy youth's proud livery so gazed on now 
  Will be a tattered weed of small worth held:
  Then being asked  where all thy beauty lies 
  Where all the treasure of thy lusty days;
  To say  within thine own deep sunken eyes 
  Were an all-eating shame  and thr

In [45]:
word_list = text.split()
a = 0
roman_num = ['ii','iii','iv','v','vi','vii','viii','ix','x','xi','xii','xiii','xiv','xv','xvi','xvii',
             'xviii','xix','xx','xxi','xxii','xxiii','xxiv','xxv','xxvi','xxvii','xxviii','xxix','xxx']
for i in range(len(word_list) - a):
    word_list[i - a] = word_list[i - a].lower().strip("“”''-.?:;!,[]~`’—&")
    if word_list[i - a] in ['nan','','*'] or word_list[i-a][0] == '&' or word_list[i-a].isdigit() \
     or word_list[i-a] in roman_num:
        word_list.pop(i - a)
        a += 1
len(word_list)

46156

In [46]:
print(word_list)

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', "beauty's", 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', "feed'st", 'thy', "light's", 'flame', 'with', 'self-substantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thyself', 'thy', 'foe', 'to', 'thy', 'sweet', 'self', 'too', 'cruel', 'thou', 'that', 'art', 'now', 'the', "world's", 'fresh', 'ornament', 'and', 'only', 'herald', 'to', 'the', 'gaudy', 'spring', 'within', 'thine', 'own', 'bud', 'buriest', 'thy', 'content', 'and', 'tender', 'churl', "mak'st", 'waste', 'in', 'niggarding', 'pity', 'the', 'world', 'or', 'else', 'this', 'glutton', 'be', 'to', 'eat', 'the', "world's", 'due', 'by', 'the', 'grave', 'and', 'thee', 'when', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow', 'and', 'dig', 'deep', 'trench

In [47]:
maxlen = 6                                                            
step = 1 

sentences = []                                                         

next_words = []                                                        

for i in range(0, len(word_list) - maxlen, step):
    sentences.append(word_list[i: i + maxlen])
    next_words.append(word_list[i + maxlen])

print('Number of sequences:', len(sentences))

words = sorted(list(set(word_list)))                                        
print('Unique words:', len(words))
word_indices = dict((word, words.index(word)) for word in words)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)      
y = np.zeros((len(sentences), len(words)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, word in enumerate(sentence):                                
        x[i, t, word_indices[word]] = 1                                
    y[i, word_indices[next_words[i]]] = 1 

Number of sequences: 46150
Unique words: 6655
Vectorization...


In [48]:
import nltk
from nltk.corpus import cmudict
nltk.download('cmudict')
d = cmudict.dict()

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\kyle_\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [49]:
# Method to add phonems for each word to 'phonem' dict
def syls(word):
    flag = False
    r_word = word[:]
    while len(r_word) > 0 and flag == False:
        try:
            phonem[word] = d[r_word.lower()]
            flag = True        # if no exception occurs then flag is set indicating phonems added to dict
        except Exception as e:
            r_word = r_word[1:]
    if r_word == '':
        phonem[word] = []

In [50]:
# initializes the phonem dict
phonem = {}
# calls the 'syls' method for each word in the 'words' list
for w in words:
    syls(w)
# phonem

In [62]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(256, input_shape=(maxlen, len(words))))
model.add(layers.Dense(len(words), activation='softmax'))

In [63]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [64]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [65]:
def rhyming_words(word):
    if word != '':
        for key in phonem:
            len_key = len(phonem[key])
            for i in range(len(phonem[word])):
                for j in range(len_key):
                    if word == key or len_key == 0:
                        continue
                    elif len(phonem[word]) == 1:
                        if phonem[word][i][-1] == phonem[key][j][-1]:
                            rhymers.append(key)
                    elif len(phonem[key]) == 2:
                        if phonem[word][i][-2:] == phonem[key][j][-2:]:
                            rhymers.append(key)
                    else:
                        if phonem[word][i][-3:] == phonem[key][j][-3:]:
                            rhymers.append(key)
#     print(len(rhymers))
    
                                           
def syllable_counter(word):
    count = 0
    if ipa.syllable_count(word) == 0:
        count = len(next_word) % 3
        if count == 0:
            count = 1
    else:
        count += ipa.syllable_count(word)
    return count

In [66]:
import random
import sys

RHYME_CHECK = 5
line_count = 0
quatrain_count = 0
quatrain = False
new_line = False
file = open("poem_output.txt", 'a')

generated_text = ''
count = 0

for epoch in range(1, 11):
    generated_text = ''
    generated_list = []
    print('\nepoch', epoch)
    hist = model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(word_list) - maxlen - 1)         
    generated_list = word_list[start_index: start_index + maxlen]

    for word in generated_list:
        generated_text += word + ' '
    generated_text =  generated_text.strip()
    print('--- Generating with seed: "' + generated_text + '"')
    for num in ipa.syllable_count(generated_text):
        count += num
    for temperature in [0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        file.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(words)))             
            for t, word in enumerate(generated_list):               
                sampled[0, t, word_indices[word]] = 1. 
                
            if count >= 10:
                sys.stdout.write('\n')
                file.write('\n')
                count = 0 
                new_line = True

            if line_count == 2 and count >= RHYME_CHECK:
                preds = model.predict(sampled, verbose=0)[0] * z1
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]
                if count + syllable_counter(next_word) >= 10:
#                         sys.stdout.write('     THIRD LINE')
                    line_count += 1 
                    if quatrain_count == 3:
                        line_count += 1
                        quatrain_count = 0
                else:
                    preds = model.predict(sampled, verbose=0)[0]
                    next_index = sample(preds, temperature)                 
                    next_word = words[next_index]
                    while (count + syllable_counter(next_word)) >= 10:
                        next_index = sample(preds, temperature)                 
                        next_word = words[next_index] 
               
            elif line_count == 3 and count >= RHYME_CHECK:
                preds = model.predict(sampled, verbose=0)[0] * z2
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]
                if (count + syllable_counter(next_word) >= 10):
#                         sys.stdout.write('     FOURTH LINE')
                    line_count += 1
                    quatrain_count += 1
                else:
                    preds = model.predict(sampled, verbose=0)[0]
                    next_index = sample(preds, temperature)                 
                    next_word = words[next_index]
                    while (count + syllable_counter(next_word)) >= 10:
                        next_index = sample(preds, temperature)                 
                        next_word = words[next_index] 

            else:
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]

            generated_list.append(next_word)
            generated_list = generated_list[1:]

            if new_line:
                next_word = next_word.capitalize()
                new_line = False
            if next_word == 'i':
                next_word = next_word.upper()
            sys.stdout.write(' ' + next_word)
            file.write(' ' + next_word)
            syl_num = syllable_counter(next_word)
            count += syl_num

            if count >= 10:
                if line_count == 0:
#                     sys.stdout.write('     FIRST LINE')
                    rhymers = []
                    z1 = np.zeros(len(words)) + 0.0001
#                     print("\nNext Word:", next_word)
                    r_word = next_word[:].lower()
                    r_word = r_word.strip(",.?!:;-'\"_")   
#                     print(r_word)
                    rhyming_words(r_word)
#                     print(rhymers)
                    for rhyme in rhymers:
                        if rhyme in word_indices:
                            z1[word_indices[rhyme]] = 10000.
                    line_count += 1
                    if quatrain_count == 3:
                        line_count += 1

                elif line_count == 1:
#                     sys.stdout.write('     SECOND LINE')
                    rhymers = []
                    z2 = np.zeros(len(words)) + 0.0001
#                     print("\nNext Word:", next_word)
                    r_word = next_word[:].lower()
                    r_word = r_word.strip(",.?!:;-'\"_")   
#                     print(r_word)
                    rhyming_words(r_word)
#                     print(rhymers)
                    for rhyme in rhymers:
                        if rhyme in word_indices:
                            z2[word_indices[rhyme]] = 10000.
                    line_count += 1

                elif line_count == 4:
                    sys.stdout.write('\n')
                    file.write('\n')
                    line_count = 0
file.close()


epoch 1
Epoch 1/1
46150/46150 [==============================] - 108s 2ms/step - loss: 6.8950
--- Generating with seed: "as pillow to thy head go"
------ temperature: 0.5
as pillow to thy head go to the the
 Of in the place of my heart and truth and
 To the once of the curse of the of bad—the
 Of my heart and me and the first of hand

 And to the while of his tongue of his might
 Make my heart and from my love to thy who
 For me I not they my love not how that
 I am not yet I I love thou art to

 Thee of my love me and not to no god
 And thy for come I sweet in it of their
 Be and and to die of the tongue of and
 Of the with violet of that of a ear

 And my poets of your love to for a art's
 Air and in my love and I am no less

 Of the and light and and to the harts of
 Words and not you do and be it of my
 Love of his beauty with the more of whereof
 To the of the days of her and and I

 I have you not not you are you I I
 Not not me I not I not her a ways
 And a breathe of all of th

 Beauty of thy heart is to me I love
 Thee as I as to thy look and thou bad—the
 Might and the sweetest look of her and give

 My heart------ temperature: 1.0
for i have sworn deep oaths in thee make thy hill love fears then
 Bell strength to rose and lest with reade hand gone

 Sound perfect without felt with lies eye con
 Hast sallow vile to onwards hair bath that level
 And began that tears should weary heaven
 Were when I gainst beholding earth in vassal

 Day up upon same left and repair translated
 A virtue to meet cold con arms terror
 Is what love might first happy lying or good
 Too hallows and not see to morning remember

 Heaven indeed joyfully arts bless sight
 God who shall to under of the world then
 Hand-in-hand the bear my love me day before that
 As thee choice dead though badges was is an

 Rosy spright and tie unmoved and being treasure
 Of the world my ill to right and thou desire

 Upon thy trinitie but is my brazen
 Enmity to become and sorrow thrall
 That at the 

 So themselves and any by compass hand
 O new no should friend I have I am wrong

 Have so poet to away me so wrong a
 Gate with prove what woo a heart soul shadow
 House so lips in thee sorrow how as sua
 Times been soul present where are overthrow

 Roses of love and if you I do not
 Gift some alone sweet poet best trust but isolate

 At turns here foul------ temperature: 1.2
love thou usest but yet be when thee now remembered
 Among indeed truth or might live I white
 Have old so looked I with me as transferred
 We are we marvellous or cries when quite

 My love sickness sin are has still made at
 Brest distant ill thee woman night me what
 Thou called truant toy-size to me barren out
 Well myself to writ the seen amor somewhat

 And longer kiss that horse soonest name do
 Toss was place a good that gone must o all
 This end pulse and well oh whether this to
 His open this am and more shall spring well

 Breathe shall be may leaves at cannot could song
 Still still again so with sma

 I part thee neere doth who dearest yet
 Did from tell your worlds put whole part when even
 Were lovely towards doth me within without
 Sleep the new this friends that which dream again

 Shall heart at those you are behold my lay
 Are againe faults way and trailing sweet steel at
 Have which a girl at sight beholding they
 Have till some beguiled heart rays
epoch 5
Epoch 1/1
46150/46150 [==============================] - 129s 3ms/step - loss: 4.4698
--- Generating with seed: "and spent the white heat pales"
------ temperature: 0.5
and spent the white heat pales
 The it which gone to doth the world the it

 Is the can make that thou men can make me
 Any way as these to be a art thou
 Art to be for thy love and still in thee
 With the breath of eyes the I shall give how

 Many a that which thou make thy place and
 Make thy love and I not love you for stand

 As the dear this shadow the dim of heart
 Upon the bid and then make it the god
 And I can true love you do I cannot
 Love him wh

 By them sweet and doth lie the bright how bad—the
 Thy it were not away the love be second

 For that her high soul to see and the day
 Night day night and night night come again time
 Do not when I love of thee I do they
 Not know but love------ temperature: 1.0
gathered smell still mussulmans and giaours not shall death nor light them

 Each and well the hath been which I know its
 Dim nature think summer form on spirits

 In some lovely mine thou youth seem hast the
 Self not for all of part or another
 Mine eyes the dearest be it and love bad—the
 Though the leave seek ways and love to strong her

 Knows is while want the conscience how thought the
 Of all own thy might be such will I shall
 Be large some strife once more them the spring farewell!—the
 Own fair moment dove we it who gentle

 Things still death sonnet where is the world the
 Oh or he only this her know so will
 In change and if wood soul translated bad—the
 Way for soul life must to the true goodwill

 Or yet if th

 Have do dead in gentle shouldst that till
 My time even young doth see or if they
 Hear the lines in time of long life since will
 Dead a lie that nor new happy nor way

 Nor knows what it god with heart treasure let
 It to see take use love come------ temperature: 1.2
my dumb thoughts speaking in effect so find up
 Only then know play save hand prove hold state
 It been against for my soul's which in sleep

 On thy above when not see in these last
 Which yet word yet love o learn knows heaven
 Not though he strength heaven for where old hast
 Thou not breath to clouds from the true power seen

 By pleasure name were smell like a live fame
 Up like well worship to bloody whole time

 White when they looked on me up heaven nay
 To o yourself been and in once given
 Work heaven unto your long pride decay
 Night you follow best like make yet self in

 His day forth dark and night by worlds through all
 Sought sworn pleasure life which thee for thy gone
 O lest them leave me bear what let

 Stay change some think with beast to then the farewell!—the
 Word that power blooms upon a face form by

 Some too own our truly too lies late side
 Each of use thou grace read long and therefore
 Fortune lean of upon thought other god
 And your till our own unto unto your

 Tis proud ill with best once true themselves perfect
 Weary had had from been
epoch 9
Epoch 1/1
46150/46150 [==============================] - 123s 3ms/step - loss: 2.6285
--- Generating with seed: "been less fair or perished young"
------ temperature: 0.5
been less fair or perished young
 Or if it were not with their desire respect

 With that we before I then can I not
 Beauty which you were so then I love me
 To love that I can then I am old but
 Long to was most one I too can the tree

 Right it and were and well thou art all those
 Art all thou in me with this my glass thy
 Will and will thy love you will not he joys
 Hast what I am should with thy love till my

 Best of love I do not love you with my
 Love t

 My mistress are they in all the purple

 That which this thy by may be I love thou
 And and I love thee and when I love you
 In that I in his sweet some bring for brow
 Of thee to me the most of all that new

 I am not all that I can for love thee
 Too I do not found in------ temperature: 1.0
in thy breast doth lie that such as did thou
 He hast the thy hand and dost by far we
 Hast clear thou such a who with thee must brow

 Bottle it and for so their he behind
 And some must last some treasure of his in
 Deep will to I care forth to long I behold
 And babe like love a I have between within

 Was myself with birds one wing seemed you in
 My close and no praises but time's within

 I your behold thy knowing thou true and that
 Thou told and clear peace about the in his
 Wide thing I to her to does thine secret
 When in silence thou thy deep now praises

 Open a heart vision best to the thy
 Sweet leave after in that thou shalt find me
 Silence not a their beauty still should sigh
 The

In [36]:
hist.history

{'loss': [1.7883135611979672, 1.0]}

In [35]:
hist.history['loss'].append(1.0)